In [11]:
from prj.agents.factory import AgentsFactory
import os
path = "/home/lorecampa/projects/jane_street_forecasting/experiments/train/lgbm/lgbm_1seeds_850_1529-0.2_20241219_131622_20241222_115340/train/model"
agent_dict = {'agent_type': 'lgbm', 'load_path': path}


agent = AgentsFactory.load_agent(agent_dict)
agent



Loading models, overwriting seeds: [1673198202]


In [12]:
from prj.config import DATA_DIR
from prj.data.data_loader import DataConfig, DataLoader

data_args = {'zero_fill': True, 'ffill': True, 'include_intrastock_norm': True}
config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)

In [13]:
X, y, w, _ = loader.load_numpy_with_partition(start_part_id=9, end_part_id=9)
X.shape, y.shape, w.shape

((6274576, 133), (6274576,), (6274576,))

In [14]:
y_pred = agent.predict(X)
y_pred.shape

(6274576,)

In [15]:
from prj.metrics import weighted_mae, weighted_mse, weighted_r2, weighted_rmse

def metrics(y_true, y_pred, weights=None):
    return {
        'r2_w': weighted_r2(y_true, y_pred, weights=weights),
        'mae_w': weighted_mae(y_true, y_pred, weights=weights),
        'mse_w': weighted_mse(y_true, y_pred, weights=weights),
        'rmse_w': weighted_rmse(y_true, y_pred, weights=weights),
    }

metrics(y, y_pred, weights=w)

{'r2_w': 0.009411799295610535,
 'mae_w': 0.5193426835427746,
 'mse_w': 0.6280173468192493,
 'rmse_w': 0.7924754550263682}

In [ ]:
# {'r2_w': 0.01070632365921409,
#  'mae_w': 0.5191565073329232,
#  'mse_w': 0.6271966387231452,
#  'rmse_w': 0.7919574727996102}

In [ ]:
# {'r2_w': 0.009936974648934394,
#  'mae_w': 0.5191110354786006,
#  'mse_w': 0.6276843939012003,
#  'rmse_w': 0.792265355737079}